In [1]:
import pandas as pd
import numpy as np
from geopy.distance import distance
import pydeck as pdk
import pickle
import requests

In [2]:
model = pickle.load(open('random_forest.pickle', 'rb'))

In [3]:
data = pd.read_csv('../../../capstone_other/files_too_large_for_github/ML_table_with_availability_clusters.csv')

In [4]:
new = data.groupby('dock_id').apply(pd.DataFrame.sample, random_state = 5, n=1).reset_index(drop=True)

In [5]:
new1 = new[['dock_id', 'latitude', 'longitude', 'tot_docks']]

In [6]:
new2 = new[['weekday_cluster', 'weekend_cluster']]

In [7]:
month = 12
day = 18
hour = 16

In [8]:
date = pd.to_datetime({'year': [2021], 'month': [month], 'day': [day], 'hour': [hour]})[0]

In [9]:
day_of_week_conversion = {0:2, 1:3, 2:4, 3:5, 4:6, 5:7, 6:1}

In [10]:
sql_day_of_week = day_of_week_conversion[date.dayofweek]

In [11]:
sql_day_of_week

7

In [12]:
month_column = np.ones(len(new), dtype=int) * month
num_day_column = np.ones(len(new), dtype=int) * sql_day_of_week
hour_column = np.ones(len(new), dtype=int) * hour
date_df = pd.DataFrame({'month': month_column, 'num_day': num_day_column, 'hour': hour_column})

In [13]:
date_df

,month,num_day,hour
0,12,7,16
1,12,7,16
2,12,7,16
3,12,7,16
4,12,7,16
...,...,...,...
324,12,7,16
325,12,7,16
326,12,7,16
327,12,7,16


In [14]:
request = requests.get('https://api.openweathermap.org/data/2.5/onecall?lat=40.7812&lon=-73.9665&exclude=minutely,current,daily,alerts&units=metric&appid=404310456b8e1c31228341dd6c95dd04')
response = request.json()
response

{'lat': 40.7812,
 'lon': -73.9665,
 'timezone': 'America/New_York',
 'timezone_offset': -18000,
 'hourly': [{'dt': 1639771200,
   'temp': 13.98,
   'feels_like': 12.01,
   'pressure': 1018,
   'humidity': 22,
   'dew_point': -6.56,
   'uvi': 0.38,
   'clouds': 1,
   'visibility': 10000,
   'wind_speed': 4.81,
   'wind_deg': 300,
   'wind_gust': 9.23,
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'clear sky',
     'icon': '01d'}],
   'pop': 0},
  {'dt': 1639774800,
   'temp': 13.7,
   'feels_like': 11.73,
   'pressure': 1018,
   'humidity': 23,
   'dew_point': -6.26,
   'uvi': 0,
   'clouds': 15,
   'visibility': 10000,
   'wind_speed': 4.32,
   'wind_deg': 304,
   'wind_gust': 9.21,
   'weather': [{'id': 801,
     'main': 'Clouds',
     'description': 'few clouds',
     'icon': '02d'}],
   'pop': 0},
  {'dt': 1639778400,
   'temp': 13,
   'feels_like': 11.03,
   'pressure': 1019,
   'humidity': 26,
   'dew_point': -5.36,
   'uvi': 0,
   'clouds': 30,
   'visibili

In [15]:
temp = response['hourly'][22]['temp']
rain = round(response['hourly'][22]['pop'])

In [16]:
weather_df = pd.DataFrame.from_dict(response['hourly'])[['dt', 'temp','pop']]
weather_df['dt'] = pd.to_datetime(weather_df['dt'], unit = 's')

In [17]:
weather_df = weather_df[weather_df['dt'] == date]

In [18]:
temp = weather_df['temp'].iloc[0]
rain = round(weather_df['pop'].iloc[0])

In [19]:
temp_column = np.ones(len(new), dtype=int) * temp
rain_column = np.ones(len(new), dtype=int) * rain

In [20]:
new1['tmp'] = temp_column
new1['rain'] = rain_column

<ipython-input-20-6b2e212f92bf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new1['tmp'] = temp_column
<ipython-input-20-6b2e212f92bf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new1['rain'] = rain_column


In [21]:
day_data = pd.concat([new1, new2], axis = 1)

In [22]:
day_data

,dock_id,latitude,longitude,tot_docks,tmp,rain,weekday_cluster,weekend_cluster
0,83,40.683826,-73.976323,62,5.92,1,2,3
1,119,40.696089,-73.978034,19,5.92,1,2,0
2,120,40.686768,-73.959282,19,5.92,1,1,0
3,143,40.692395,-73.993379,24,5.92,1,2,3
4,144,40.698399,-73.980689,19,5.92,1,2,0
...,...,...,...,...,...,...,...,...
324,3505,40.805726,-73.936322,33,5.92,1,2,2
325,3506,40.801307,-73.939817,29,5.92,1,0,2
326,3507,40.804555,-73.939686,36,5.92,1,0,2
327,3509,40.801194,-73.950074,31,5.92,1,2,0


In [23]:
final_df = pd.concat([date_df, day_data], axis = 1)

In [24]:
dock_id_series = final_df['dock_id']

In [25]:
final_df.drop(['dock_id'], axis = 1, inplace = True)

In [26]:
final_df

,month,num_day,hour,latitude,longitude,tot_docks,tmp,rain,weekday_cluster,weekend_cluster
0,12,7,16,40.683826,-73.976323,62,5.92,1,2,3
1,12,7,16,40.696089,-73.978034,19,5.92,1,2,0
2,12,7,16,40.686768,-73.959282,19,5.92,1,1,0
3,12,7,16,40.692395,-73.993379,24,5.92,1,2,3
4,12,7,16,40.698399,-73.980689,19,5.92,1,2,0
...,...,...,...,...,...,...,...,...,...,...
324,12,7,16,40.805726,-73.936322,33,5.92,1,2,2
325,12,7,16,40.801307,-73.939817,29,5.92,1,0,2
326,12,7,16,40.804555,-73.939686,36,5.92,1,0,2
327,12,7,16,40.801194,-73.950074,31,5.92,1,2,0


In [27]:
predictions = model.best_estimator_.predict(final_df)

In [28]:
final_df['predictions'] = predictions
final_df['dock_id'] = dock_id_series


In [29]:
final_df['avail_bikes_proportion'] = final_df['predictions']/final_df['tot_docks']

In [30]:
final_df

,month,num_day,hour,latitude,longitude,tot_docks,tmp,rain,weekday_cluster,weekend_cluster,predictions,dock_id,avail_bikes_proportion
0,12,7,16,40.683826,-73.976323,62,5.92,1,2,3,27.830707,83,0.448882
1,12,7,16,40.696089,-73.978034,19,5.92,1,2,0,11.117740,119,0.585144
2,12,7,16,40.686768,-73.959282,19,5.92,1,1,0,5.459364,120,0.287335
3,12,7,16,40.692395,-73.993379,24,5.92,1,2,3,8.903558,143,0.370982
4,12,7,16,40.698399,-73.980689,19,5.92,1,2,0,11.008241,144,0.579381
...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,12,7,16,40.805726,-73.936322,33,5.92,1,2,2,12.025136,3505,0.364398
325,12,7,16,40.801307,-73.939817,29,5.92,1,0,2,3.511371,3506,0.121082
326,12,7,16,40.804555,-73.939686,36,5.92,1,0,2,26.770629,3507,0.743629
327,12,7,16,40.801194,-73.950074,31,5.92,1,2,0,22.447961,3509,0.724128


In [31]:
def manhattan_distance(start_lat, start_lon, end_lat, end_lon):
    dist = distance((start_lat, start_lon), (start_lat, end_lon)).miles + \
           distance((end_lat, end_lon), (start_lat, end_lon)).miles
    return dist

In [108]:
low_bike_threshold = 1/3
high_bike_threshold = 2/3
max_distance = 2
max_bikes_rebalanced = 100
min_cargo_size = 3

In [109]:
data_low = final_df[final_df['avail_bikes_proportion'] <= low_bike_threshold]

In [110]:
data_high = final_df[final_df['avail_bikes_proportion'] >= high_bike_threshold]

In [111]:
data_low['deficit'] = round((low_bike_threshold - data_low['avail_bikes_proportion']) * data_low['tot_docks']).astype('int')

<ipython-input-111-d746bb208678>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_low['deficit'] = round((low_bike_threshold - data_low['avail_bikes_proportion']) * data_low['tot_docks']).astype('int')


In [112]:
data_high['surplus'] = round((data_high['avail_bikes_proportion'] - high_bike_threshold) * data_high['tot_docks']).astype('int')

<ipython-input-112-d9c66eeb9a9d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_high['surplus'] = round((data_high['avail_bikes_proportion'] - high_bike_threshold) * data_high['tot_docks']).astype('int')


In [113]:
data_low = data_low.sort_values(by = 'deficit', ascending = False)

In [114]:
data_high = data_high.sort_values(by = 'surplus', ascending = False)

In [115]:
rebalancing_dict = {}

In [116]:
low_copy = data_low.copy()
high_copy = data_high.copy()

In [117]:
bikes_rebalanced = 0
for low in low_copy.index:
    if low_copy.loc[low, 'deficit'] == 0:
        continue
    for high in high_copy.index:
        if high_copy.loc[high, 'surplus'] == 0:
            continue
        if manhattan_distance(low_copy.loc[low, 'latitude'], low_copy.loc[low, 'longitude'], 
                              high_copy.loc[high, 'latitude'], high_copy.loc[high, 'longitude']) < max_distance:
            stations_key = (low_copy.loc[low, 'dock_id'], high_copy.loc[high, 'dock_id'])
            change = min(low_copy.loc[low, 'deficit'], high_copy.loc[high, 'surplus'])
            low_copy.loc[low, 'deficit'] -= change
            high_copy.loc[high, 'surplus'] -= change
            bikes_rebalanced += change
            if stations_key in rebalancing_dict.keys():
                rebalancing_dict[stations_key] += change
            else:
                rebalancing_dict[stations_key] = change
            if low_copy.loc[low, 'deficit'] == 0:
                break

In [118]:
rebalancing_dict

{(3161, 3318): 9,
 (3161, 3158): 5,
 (469, 3158): 1,
 (469, 3165): 5,
 (469, 3137): 5,
 (469, 3356): 2,
 (164, 3163): 1,
 (3457, 3356): 2,
 (3437, 3374): 9,
 (3437, 3387): 3,
 (3141, 3296): 5,
 (3141, 3312): 4,
 (3226, 3387): 2,
 (3301, 3500): 6,
 (3301, 3509): 2,
 (3294, 3497): 2,
 (3462, 334): 2,
 (3320, 3507): 3,
 (3479, 3342): 7,
 (3479, 3409): 2,
 (195, 3467): 9,
 (3127, 3123): 5,
 (3127, 3109): 2,
 (524, 168): 8,
 (3418, 3324): 8,
 (3075, 539): 7,
 (528, 168): 2,
 (528, 3224): 5,
 (2012, 3224): 4,
 (2012, 336): 3,
 (448, 336): 5,
 (448, 462): 1,
 (3452, 2002): 6,
 (224, 3427): 6,
 (3384, 3410): 5,
 (3310, 3410): 2,
 (3310, 3429): 3,
 (406, 355): 4,
 (491, 3427): 2,
 (491, 368): 2,
 (3306, 3429): 4,
 (3050, 3046): 3,
 (3050, 3070): 1,
 (3449, 2002): 1,
 (3449, 3070): 2,
 (3358, 310): 3,
 (3354, 3419): 2,
 (167, 462): 2,
 (3304, 310): 2,
 (245, 310): 2,
 (249, 368): 2,
 (3102, 3067): 1,
 (389, 296): 2,
 (3073, 3241): 2,
 (418, 355): 2,
 (2005, 3414): 1,
 (313, 3419): 1,
 (3064, 437

In [119]:
sorted_rebalancing = dict(sorted(rebalancing_dict.items(), key=lambda x: x[1], reverse = True))

In [120]:
filtered_rebalancing = {key: value for key, value in sorted_rebalancing.items() if value >= min_cargo_size}

In [121]:
filtered_rebalancing

{(3161, 3318): 9,
 (3437, 3374): 9,
 (195, 3467): 9,
 (524, 168): 8,
 (3418, 3324): 8,
 (3479, 3342): 7,
 (3075, 539): 7,
 (3301, 3500): 6,
 (3452, 2002): 6,
 (224, 3427): 6,
 (3161, 3158): 5,
 (469, 3165): 5,
 (469, 3137): 5,
 (3141, 3296): 5,
 (3127, 3123): 5,
 (528, 3224): 5,
 (448, 336): 5,
 (3384, 3410): 5,
 (3141, 3312): 4,
 (2012, 3224): 4,
 (406, 355): 4,
 (3306, 3429): 4,
 (3437, 3387): 3,
 (3320, 3507): 3,
 (2012, 336): 3,
 (3310, 3429): 3,
 (3050, 3046): 3,
 (3358, 310): 3}

In [122]:
filtered_bikes_rebalanced = 0
final_rebalancing_dict = {}
for k, v in filtered_rebalancing.items():
    if filtered_bikes_rebalanced < max_bikes_rebalanced:
        final_rebalancing_dict[k] = v
        filtered_bikes_rebalanced += v

In [123]:
rebalancing_df = pd.DataFrame(final_rebalancing_dict.items(), columns = ['dock_ids', 'num_bikes'])

In [124]:
rebalancing_df[['dock_id_receive', 'dock_id_give']] = rebalancing_df['dock_ids'].tolist()
rebalancing_df.drop(['dock_ids'], axis = 1, inplace = True)

In [125]:
data_df = final_df[['dock_id', 'latitude', 'longitude']]

In [126]:
rebalancing_df = rebalancing_df.merge(data_df, how = 'left', left_on = 'dock_id_receive', right_on = 'dock_id').rename(
    columns = {'latitude': 'latitude_receive', 'longitude': 'longitude_receive'})
rebalancing_df = rebalancing_df.merge(data_df, how = 'left', left_on = 'dock_id_give', right_on = 'dock_id').rename(
    columns = {'latitude': 'latitude_give', 'longitude': 'longitude_give'})
rebalancing_df.drop(['dock_id_x', 'dock_id_y'], axis = 1, inplace = True)

In [127]:
GREEN_RGB = [0, 255, 0, 150]
RED_RGB = [240, 100, 0, 150]

# Specify a deck.gl ArcLayer
arc_layer = pdk.Layer(
    "ArcLayer",
    data = rebalancing_df,
    get_width="num_bikes",
    get_source_position=["longitude_give", "latitude_give"],
    get_target_position=["longitude_receive", "latitude_receive"],
    get_tilt=15,
    get_source_color=RED_RGB,
    get_target_color=GREEN_RGB,
    pickable=True,
    auto_highlight=True,
)

view_state = pdk.ViewState(latitude=40.74, longitude=-74, bearing=45, pitch=50, zoom=8,)


TOOLTIP_TEXT = {"html": "{num_bikes} rebalanced"}
r = pdk.Deck(arc_layer, initial_view_state=view_state, tooltip=TOOLTIP_TEXT, map_style = 'light')
r

{"initialViewState": {"bearing": 45, "latitude": 40.74, "longitude": -74, "pitch": 50, "zoom": 8}, "layers": [{"@@type": "ArcLayer", "autoHighlight": true, "data": [{"dock_id_give": 3318, "dock_id_receive": 3161, "latitude_give": 40.7839636, "latitude_receive": 40.7801839642592, "longitude_give": -73.9471673, "longitude_receive": -73.97728532355958, "num_bikes": 9}, {"dock_id_give": 3374, "dock_id_receive": 3437, "latitude_give": 40.799484002827455, "latitude_receive": 40.79313480986663, "longitude_give": -73.95561300212057, "longitude_receive": -73.97700369358063, "num_bikes": 9}, {"dock_id_give": 3467, "dock_id_receive": 195, "latitude_give": 40.72494672359416, "latitude_receive": 40.70905623, "longitude_give": -74.00165855884552, "longitude_receive": -74.01043382, "num_bikes": 9}, {"dock_id_give": 168, "dock_id_receive": 524, "latitude_give": 40.73971301, "latitude_receive": 40.75527307, "longitude_give": -73.99456405, "longitude_receive": -73.98316936, "num_bikes": 8}, {"dock_id_give": 3324, "dock_id_receive": 3418, "latitude_give": 40.6685455, "latitude_receive": 40.6750207, "longitude_give": -73.99333264, "longitude_receive": -73.97111473, "num_bikes": 8}, {"dock_id_give": 3342, "dock_id_receive": 3479, "latitude_give": 40.6777748, "latitude_receive": 40.68512559515416, "longitude_give": -74.0094613, "longitude_receive": -74.02535319328307, "num_bikes": 7}, {"dock_id_give": 539, "dock_id_receive": 3075, "latitude_give": 40.71534825, "latitude_receive": 40.70708701, "longitude_give": -73.96024116, "longitude_receive": -73.95796783, "num_bikes": 7}, {"dock_id_give": 3500, "dock_id_receive": 3301, "latitude_give": 40.7989445, "latitude_receive": 40.79195568586411, "longitude_give": -73.944846, "longitude_receive": -73.96808699871315, "num_bikes": 6}, {"dock_id_give": 2002, "dock_id_receive": 3452, "latitude_give": 40.71688699585005, "latitude_receive": 40.71915571298016, "longitude_give": -73.96319799439499, "longitude_receive": -73.94885393926462, "num_bikes": 6}, {"dock_id_give": 3427, "dock_id_receive": 224, "latitude_give": 40.72430527250332, "latitude_receive": 40.71146364, "longitude_give": -73.99600982666014, "longitude_receive": -74.00552427, "num_bikes": 6}, {"dock_id_give": 3158, "dock_id_receive": 3161, "latitude_give": 40.77163852214548, "latitude_receive": 40.7801839642592, "longitude_give": -73.98261428344092, "longitude_receive": -73.97728532355958, "num_bikes": 5}, {"dock_id_give": 3165, "dock_id_receive": 469, "latitude_give": 40.775793063619005, "latitude_receive": 40.76344058, "longitude_give": -73.97620285768829, "longitude_receive": -73.98268129, "num_bikes": 5}, {"dock_id_give": 3137, "dock_id_receive": 469, "latitude_give": 40.77282817194925, "latitude_receive": 40.76344058, "longitude_give": -73.96685277301444, "longitude_receive": -73.98268129, "num_bikes": 5}, {"dock_id_give": 3296, "dock_id_receive": 3141, "latitude_give": 40.782454175279874, "latitude_receive": 40.76500524724615, "longitude_give": -73.9489196240902, "longitude_receive": -73.9581849122329, "num_bikes": 5}, {"dock_id_give": 3123, "dock_id_receive": 3127, "latitude_give": 40.744697437987, "latitude_receive": 40.74966, "longitude_give": -73.93540373202681, "longitude_receive": -73.9521, "num_bikes": 5}], "getSourceColor": [240, 100, 0, 150], "getSourcePosition": "@@=[longitude_give, latitude_give]", "getTargetColor": [0, 255, 0, 150], "getTargetPosition": "@@=[longitude_receive, latitude_receive]", "getTilt": 15, "getWidth": "@@=num_bikes", "id": "989ca405-25ac-47b1-adc3-5052b1e7072d", "pickable": true}], "mapProvider": "carto", "mapStyle": "https://basemaps.cartocdn.com/gl/positron-gl-style/style.json", "views": [{"@@type": "MapView", "controller": true}]}

In [128]:
bikes_rebalanced

203

In [129]:
filtered_bikes_rebalanced

100

In [130]:
data_low['deficit'].sum()

647

In [131]:
data_high['surplus'].sum()

391

In [106]:
data_low.head(5)

,month,num_day,hour,latitude,longitude,tot_docks,tmp,rain,weekday_cluster,weekend_cluster,predictions,dock_id,avail_bikes_proportion,deficit
151,12,7,16,40.780184,-73.977285,59,5.92,1,1,2,5.417160,3161,0.091816,14
79,12,7,16,40.763441,-73.982681,57,5.92,1,0,1,5.849455,469,0.102622,13
6,12,7,16,40.753231,-73.970325,47,5.92,1,5,1,3.728339,164,0.079326,12
297,12,7,16,40.763026,-73.972095,46,5.92,1,5,1,3.320936,3457,0.072194,12
138,12,7,16,40.763126,-73.965269,51,5.92,1,0,1,4.525877,3134,0.088743,12


In [107]:
data_high.head(5)

,month,num_day,hour,latitude,longitude,tot_docks,tmp,rain,weekday_cluster,weekend_cluster,predictions,dock_id,avail_bikes_proportion,surplus
8,12,7,16,40.739713,-73.994564,47,5.92,1,5,3,41.067906,168,0.873785,10
303,12,7,16,40.724947,-74.001659,42,5.92,1,2,0,37.488517,3467,0.892584,9
244,12,7,16,40.799484,-73.955613,36,5.92,1,3,3,33.129846,3374,0.920274,9
200,12,7,16,40.783964,-73.947167,39,5.92,1,3,0,35.177377,3318,0.901984,9
164,12,7,16,40.739974,-74.005139,39,5.92,1,2,0,34.983376,3224,0.897010,9
